## Introduction

Is is possible to predict the gender of a character in a story using the pronouns around them?

Yes

He and she are objective pronouns

Him and her are subjective pronouns

His and her are possessive adjectives

His and hers are possessive pronouns

"Her" is possessive when followed by a noun.

## Methodology

## Hypothesis



In [ ]:
%pip install sparknlp
%pip install pyspark
%pip install spacy
import sparknlp
import os
import spacy
import pyspark
from pyspark.sql import SparkSession
from tqdm import tqdm
spark = sparknlp.start()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.2/579.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=85bd62732d1bdb7677fa42ef675f48e787fcf15a7f37812383c2bbdbdf441aa8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from sparknlp.pretrained import PretrainedPipeline
root = os.path.dirname(os.path.realpath('sparknlp.ipynb'))
file = open(root + '/sample_data/pg1342.txt',"r")
pride = file.read()

In [ ]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


In [ ]:
#result = pipeline.annotate(pride)

In [ ]:
# tok_tag = [(result['token'], result['pos'])]
# zips = [list(zip(tt[0],tt[1])) for tt in tok_tag]

NameError: name 'result' is not defined

In [ ]:
pronouns = ['he','him','his','she','her','hers']
def pronoun_check(x):
  '''
  x = list of tuples
  returns list of tuples with only pronouns
  '''
  pro_toks = []
  male_count, female_count = 0, 0
  for i in x:
    flag = False
    last_pos = ''
    last_tok = ''
    for y in i:
      if flag:
        if pos != 'NN' and pos != 'NNS' and token == '.':
          female_count += 1
          flag = False
          continue
      token, pos = y
      token = token.lower()
      # Magic (identifies instances of 'her' that are objective)
      if token in pronouns:
        if token == 'her':
          flag = True
          last_pos = pos
          last_tok = token
        else:
            if token == 'he' or token == 'him':
              male_count += 1
            else:
              female_count += 1

  return male_count, female_count

#def count_pronouns(x):

In [ ]:
class Node:
  def __init__(self, data):
    # reference to next node in DLL
    self.next = None
    # reference to previous node in DLL
    self.prev = None
    self.data = data

def insert_at_end(head, data):
  # insert new node at the end of the list
  new_node = Node(data)
  if head is None:
    return new_node

  current = head
  while current.next:
    current = current.next

  current.next = new_node
  new_node.prev = current
  return head

def display(head):
  current = head
  while current:
    print(current.data, end = " <-> ")
    current = current.next
  print(None)

In [ ]:
head = None
head = insert_at_end(head, 1)
head = insert_at_end(head, 2)
head = insert_at_end(head, 3)

print("Doubly Linked List after insertion at the end:")
display(head)

Doubly Linked List after insertion at the end:
1 <-> 2 <-> 3 <-> None


In [ ]:
pride = pride.replace('Mr.','Mr')
pride = pride.replace('Mrs.','Mrs')
pride = pride.replace('Ms.','Ms')
pride = pride.replace('Dr.','Dr')
pride = pride.replace('_','')
pride = pride.replace('-','')
pridesplit = pride.split('.')

In [ ]:
def period(sentences):
  sent_list = []
  for sent in sentences:
    sent = sent+'.'
    sent_list.append(sent)
  return sent_list

In [ ]:
pridesplit = period(pridesplit)

In [ ]:
for sent in pridesplit:
  results = []
  result = pipeline.annotate(sent)
  results.append(result)
return results

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.10/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

for each sentence, find all people mentiond. Add the number of male and female pronounce counted in the sentence to a dict where the key is the persons name.

In [ ]:
nlp = spacy.load('en_core_web_sm')
personGenderCount = {} # person: malePronounCount, femalePronounCount
# for sent in pridesplit:
for sent in tqdm(pridesplit):
  doc = nlp(sent)
  people = []
  for ent in doc.ents:
    if ent.label_ == 'PERSON':
      people.append(ent.text)
  if len(people) == 0:
    continue
  annoted_sent = pipeline.annotate(sent)
  tok_tag = [(annoted_sent['token'], annoted_sent['pos'])]
  zips = [list(zip(tt[0],tt[1])) for tt in tok_tag]
  male_count, female_count = pronoun_check(zips)
  for person in people:
    try:
      person_male_count = personGenderCount[person][0]
      person_female_count = personGenderCount[person][1]
      personGenderCount[person] = person_male_count + male_count, person_female_count + female_count
    except KeyError:
      personGenderCount[person] = male_count, female_count


personGenderCount


100%|██████████| 5516/5516 [12:49<00:00,  7.16it/s]


{'Project Gutenberg eBook': (0, 0),
 'gutenberg': (0, 0),
 'Jane Austen': (0, 0),
 'Chuck Greif': (0, 0),
 'GEORGE ALLEN\n                               ': (0, 0),
 'George Saintsbury': (0, 0),
 'Hugh Thomson': (0, 1),
 'George Allen': (1, 5),
 'CHANCERY LANE': (0, 0),
 'Comyns Carr': (0, 1),
 'Walt Whitman': (0, 0),
 'Mary': (10, 26),
 'Crawford': (2, 0),
 'Mrs Norris': (2, 0),
 'Emma': (0, 0),
 'Austen': (3, 11),
 'Miss Bates': (0, 0),
 'Miss Austen': (0, 0),
 'Wickham': (64, 86),
 'Mrs Rushworth': (0, 0),
 'Jane': (139, 287),
 'Frank Churchill': (0, 0),
 'Jane Fairfax': (0, 0),
 'Darcy': (276, 431),
 'Elizabeth': (242, 512),
 'John\nThorpe': (0, 0),
 'Swift': (0, 0),
 'Smollett': (0, 0),
 'Miss Burney': (0, 0),
 'Addison': (0, 0),
 'Lydia Bennet': (0, 1),
 'Mrs Musgrove’s': (0, 0),
 'Bennet': (27, 53),
 'Goldwin Smith': (1, 0),
 'Memling': (0, 1),
 'Meissonier': (0, 1),
 'Mary Wollstonecraft': (0, 0),
 'Collins': (52, 78),
 'Mrs\nBennet': (3, 8),
 'Mrs Bennet': (36, 66),
 'Thackeray

convert dict to list so we can itterate through it

In [17]:
# convert personGenderCount to list
personGenderList = []
for key, value in personGenderCount.items():
  m_count, f_count = value[0], value[1]
  if m_count + f_count < 10:
    continue
  personGenderList.append([key, value[0], value[1]])


manually verify each persons gender (do not presume to know unless prefaced by Mr, Mrs, ... etc)

In [18]:
out_list = [] # will consist of [name, prediction, actual]
for person in personGenderList:
  name, male_count, female_count = person[0], person[1], person[2]
  prediciton = 1 if male_count > female_count else 0
  actual = input('Is ' + name + ': ' + str(prediciton) + '? ') # 1 for M, 0 for F
  out_list.append([name, prediciton, actual])

Is Mary: 0? 0
Is Austen: 0? 0
Is Wickham: 0? 1
Is Jane: 0? 0
Is Darcy: 0? 1
Is Elizabeth: 0? 0
Is Bennet: 0? 0
Is Collins: 0? 1
Is Mrs
Bennet: 0? 0
Is Mrs Bennet: 0? 0
Is Mr Collins: 0? 1
Is Lady Catherine de Bourgh: 0? 0
Is Lady Catherine: 0? 0
Is Kitty: 0? 0
Is Mrs Bennet’s: 0? 0
Is Bingleys: 0? 1
Is Pemberley: 0? -1
Is Mr Bingley: 0? 1
Is Lizzy: 0? 0
Is Mrs Long: 1? 0
Is Bingley: 0? 1
Is William: 1? 1
Is Lydia: 0? 0
Is Mrs Hurst: 0? 0
Is Catherine: 0? 0
Is Miss Bingley: 0? 0
Is Miss Bennet: 0? 0
Is William Lucas: 0? 1
Is Lucas: 0? 0
Is Robinson: 1? 1
Is Forster: 1? 1
Is Meryton: 1? -1
Is Mrs Philips: 0? 0
Is Charles: 1? 1
Is Netherfield: 0? -1
Is Gardiner: 1? 1
Is Miss de Bourgh: 0? 0
Is Denny: 1? 1
Is Mr Wickham: 0? 0
Is Lady
Catherine: 0? 0
Is Rosings: 0? -1
Is De Bourgh: 0? 0
Is George Wickham: 1? 1
Is Mrs Gardiner: 0? 0
Is Kent: 0? -1
Is Maria: 0? 0
Is Mrs Collins: 0? 0
Is Mrs Collins’s: 0? 0
Is Fitzwilliam: 0? 1
Is Mrs Forster: 0? 0
Is Mrs Reynolds: 0? 0


save to file

In [31]:
import pandas as pd
import datetime
df = pd.DataFrame(out_list, columns=['name', 'prediction', 'actual'])
df = df.astype({'actual': int})
df.to_csv('output_' + datetime.datetime.now().strftime("%Y-%m-%d") + '.csv', index=False)
df = df[df['actual'] != -1] # drop locations, after filesave (for scoring)


In [32]:
df

,name,prediction,actual
0,Mary,0,0
1,Austen,0,0
2,Wickham,0,1
3,Jane,0,0
4,Darcy,0,1
5,Elizabeth,0,0
6,Bennet,0,0
7,Collins,0,1
8,Mrs\nBennet,0,0
9,Mrs Bennet,0,0


calculate accuracy

In [38]:
# calculate accuracy, when predicted == actual that's good
accuracy = (df['prediction'] == df['actual']).sum() / len(df)
print(f'Accuracy: {round(accuracy*100,2)}%')

Accuracy: 78.26%
